In [3]:
%%time
import requests
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta

CPU times: user 1.3 s, sys: 541 ms, total: 1.85 s
Wall time: 6.83 s


In [4]:
%%time
# read as DataFrame
select_df = pd.read_csv("../Data/select_milkrun_order_data.csv")
print(select_df.shape)
select_df.head()

(19473, 25)
CPU times: user 120 ms, sys: 23.5 ms, total: 143 ms
Wall time: 1.54 s


,Unnamed: 0,merchant_id,merchant_name,driver_id,driver_name,vehicle_type,order_id,pick_up_address,delivery_address,paid_fee,...,picked_at,delivered_at,cancelled_at,driver_paid_fee,car_only,tips,Restaurant Area,pick_up_coordinate,delivery_coordinate,coordinate_distance
0,136,39,CSC,186.0,Zoey MilkRun,1.0,CS0001,"21 Amber Rd, Singapore 439870","3 Haig Rd, Singapore 430003",5.0,...,2021-01-12 15:45:20.555,2021-01-12 15:46:15.883,NaN,4.25,False,0.0,NaN,"(1.299306492, 103.9005769)","(1.31170259, 103.8973795)",1.423476
1,200,41,CSC Manzhu Cafe,12.0,mars rawi,0.0,CS0010,"21 Amber Rd, Singapore 439870",Singapore 440047,3.5,...,NaN,NaN,2021-01-27 13:03:44.777,2.75,False,0.0,NaN,"(1.299306492, 103.9005769)","(1.305723545, 103.9125305)",1.508296
2,201,41,CSC Manzhu Cafe,12.0,mars rawi,0.0,CS0011,"21 Amber Rd, Singapore 439870",Singapore 440047,3.5,...,2021-01-27 13:32:34.059,2021-01-27 13:46:32.818,NaN,2.75,False,0.0,NaN,"(1.299306492, 103.9005769)","(1.305723545, 103.9125305)",1.508296
3,205,41,CSC Manzhu Cafe,12.0,mars rawi,0.0,CS0014,"21 Amber Rd, Singapore 439870","3 Haig Rd, Block 3, Singapore 430003",3.5,...,2021-01-27 18:47:17.782,2021-01-27 18:59:33.035,NaN,2.75,False,0.0,NaN,"(1.299306492, 103.9005769)","(1.31170259, 103.8973795)",1.423476
4,212,41,CSC Manzhu Cafe,NaN,NaN,NaN,CS0028,"21 Amber Rd, Singapore 439870","3 Haig Rd, Singapore 430003",3.5,...,NaN,NaN,2021-01-28 20:20:51.027,NaN,False,0.0,NaN,"(1.299306492, 103.9005769)","(1.31170259, 103.8973795)",1.423476


In [5]:
order_per_time_period = pd.read_csv("../Data/order_per_time_period_2.csv")
order_per_time_period.columns = ['time_period_id', 'create_time_period', 'number']
order_per_time_period

,time_period_id,create_time_period,number
0,0,2021-10-10 06:00:00,5
1,1,2021-10-10 06:45:00,15
2,2,2021-10-10 07:30:00,22
3,3,2021-10-10 08:15:00,34
4,4,2021-10-10 09:00:00,47
5,5,2021-10-10 09:45:00,63
6,6,2021-10-10 10:30:00,70
7,7,2021-10-10 11:15:00,60
8,8,2021-10-10 12:00:00,35
9,9,2021-10-10 12:45:00,28


In [6]:
# convert to dict
order_per_time_period_dict = pd.Series(order_per_time_period.number.values, index=order_per_time_period.time_period_id).to_dict()
order_per_time_period_dict

{0: 5,
 1: 15,
 2: 22,
 3: 34,
 4: 47,
 5: 63,
 6: 70,
 7: 60,
 8: 35,
 9: 28,
 10: 22,
 11: 24,
 12: 26,
 13: 40,
 14: 56,
 15: 65,
 16: 56,
 17: 46,
 18: 47,
 19: 39,
 20: 32,
 21: 8}

In [7]:
time_id_period_dict = pd.Series(order_per_time_period.create_time_period.values, index=order_per_time_period.time_period_id).to_dict()
time_id_period_dict

{0: '2021-10-10 06:00:00',
 1: '2021-10-10 06:45:00',
 2: '2021-10-10 07:30:00',
 3: '2021-10-10 08:15:00',
 4: '2021-10-10 09:00:00',
 5: '2021-10-10 09:45:00',
 6: '2021-10-10 10:30:00',
 7: '2021-10-10 11:15:00',
 8: '2021-10-10 12:00:00',
 9: '2021-10-10 12:45:00',
 10: '2021-10-10 13:30:00',
 11: '2021-10-10 14:15:00',
 12: '2021-10-10 15:00:00',
 13: '2021-10-10 15:45:00',
 14: '2021-10-10 16:30:00',
 15: '2021-10-10 17:15:00',
 16: '2021-10-10 18:00:00',
 17: '2021-10-10 18:45:00',
 18: '2021-10-10 19:30:00',
 19: '2021-10-10 20:15:00',
 20: '2021-10-10 21:00:00',
 21: '2021-10-10 21:45:00'}

In [8]:
time_period_id = list(order_per_time_period_dict.keys())
time_period_id

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [9]:
import csv

with open('../Data/restaurant_list.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

print(data)

[['Ghim Moh Market & Food Centre', 'Maxwell Food Centre', 'Hong Lim Food Centre', 'Old Airport Road Food Centre', '219 Bedok Central, Block 219, Êñ∞Âä†Âù° 460219', 'Kovan Food Centre (209 Hougang Street 21)', 'Bedok Interchange Hawker Centre', 'Bukit Timah Market & Food Centre', '37 Cuppage Rd, Singapore 229460, Singapore', '21 Amber Rd, Singapore 439870', 'Alexandra Village Food Centre', 'Tiong Bahru Market', '632 Bedok Reservoir Rd, Singapore 470632, Singapore', 'Lorong 8 Toa Payoh Hawker Centre', 'Bukit Merah View Market & Hawker Centre', 'Amoy Street Food Centre', 'Golden Mile Food Centre', 'Chomp Chomp', '66 Kg Bugis, Singapore 338987, Singapore', 'Breadtalk Ihq, 30 Tai Seng St, Singapore 534013, Singapore', 'Zion Riverside Food Centre', '36 Boat Quay, Singapore 049825, Singapore', '28 Maju Ave, Singapore 556698', '417 River Valley Rd, Singapore 248316, Singapore', 'Our Tampines Hub Hawker Centre', '371 Beach Rd, Singapore 199597, Singapore', '54 South Bridge Rd, Singapore 058685,

In [9]:
restaurant_list = data[0]
restaurant_list

['Ghim Moh Market & Food Centre',
 'Maxwell Food Centre',
 'Hong Lim Food Centre',
 'Old Airport Road Food Centre',
 '219 Bedok Central, Block 219, Êñ∞Âä†Âù° 460219',
 'Kovan Food Centre (209 Hougang Street 21)',
 'Bedok Interchange Hawker Centre',
 'Bukit Timah Market & Food Centre',
 '37 Cuppage Rd, Singapore 229460, Singapore',
 '21 Amber Rd, Singapore 439870',
 'Alexandra Village Food Centre',
 'Tiong Bahru Market',
 '632 Bedok Reservoir Rd, Singapore 470632, Singapore',
 'Lorong 8 Toa Payoh Hawker Centre',
 'Bukit Merah View Market & Hawker Centre',
 'Amoy Street Food Centre',
 'Golden Mile Food Centre',
 'Chomp Chomp',
 '66 Kg Bugis, Singapore 338987, Singapore',
 'Breadtalk Ihq, 30 Tai Seng St, Singapore 534013, Singapore',
 'Zion Riverside Food Centre',
 '36 Boat Quay, Singapore 049825, Singapore',
 '28 Maju Ave, Singapore 556698',
 '417 River Valley Rd, Singapore 248316, Singapore',
 'Our Tampines Hub Hawker Centre',
 '371 Beach Rd, Singapore 199597, Singapore',
 '54 South Bri

In [26]:
import json
 
# Opening JSON file
f = open('../Data/restaurant_customer_dict.json', 'r')
 
# returns JSON object as
# a dictionary
restaurant_customer_dict = json.loads(f.read())


restaurant_customer_dict

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
def restaurant_seed_generation(all_restaurants):
    '''
    Input:
    - all_restaurant: predefined instance: restaurant_list
    Output:
    - dict, {restaurant: random seed (integer value)}
    '''
    seed_num_list = random.sample(range(1, 100), len(all_restaurants))
    restaurant_seed_dict = {all_restaurants[i]: seed_num_list[i] for i in range(len(all_restaurants))}
    
    return restaurant_seed_dict

In [11]:
restaurant_seed_num_dict = restaurant_seed_generation(restaurant_list)
restaurant_seed_num_dict

{'Ghim Moh Market & Food Centre': 6,
 'Maxwell Food Centre': 46,
 'Hong Lim Food Centre': 72,
 'Old Airport Road Food Centre': 35,
 '219 Bedok Central, Block 219, Êñ∞Âä†Âù° 460219': 4,
 'Kovan Food Centre (209 Hougang Street 21)': 79,
 'Bedok Interchange Hawker Centre': 99,
 'Bukit Timah Market & Food Centre': 56,
 '37 Cuppage Rd, Singapore 229460, Singapore': 92,
 '21 Amber Rd, Singapore 439870': 22,
 'Alexandra Village Food Centre': 29,
 'Tiong Bahru Market': 96,
 '632 Bedok Reservoir Rd, Singapore 470632, Singapore': 80,
 'Lorong 8 Toa Payoh Hawker Centre': 20,
 'Bukit Merah View Market & Hawker Centre': 97,
 'Amoy Street Food Centre': 60,
 'Golden Mile Food Centre': 48,
 'Chomp Chomp': 86,
 '66 Kg Bugis, Singapore 338987, Singapore': 3,
 'Breadtalk Ihq, 30 Tai Seng St, Singapore 534013, Singapore': 89,
 'Zion Riverside Food Centre': 50,
 '36 Boat Quay, Singapore 049825, Singapore': 31,
 '28 Maju Ave, Singapore 556698': 95,
 '417 River Valley Rd, Singapore 248316, Singapore': 30,
 '

In [12]:
def order_num_generation(restaurants):
    '''
    Generate number of orders for each restaurant at each time period
    Input:
    - restaurants: list of restaurant
    Output:
    - dict {restaurant: [time period, number of orders]}
    '''
    restaurant_order_num = {r: [] for r in restaurants}
    
    for r in restaurants:
        # get restaurant seed number 
        seed_num = restaurant_seed_num_dict[r]
        random.seed(seed_num)
        alpha = random.uniform(0.5, 1.5)
        for t in time_period_id:
            order_num = int(order_per_time_period_dict[t] * alpha) + random.randint(0, 5)
            restaurant_order_num[r].append([t, order_num])
    
    return restaurant_order_num

In [13]:
# test
restaurant_order_num = order_num_generation(restaurant_list)
restaurant_order_num

{'Ghim Moh Market & Food Centre': [[0, 6],
  [1, 22],
  [2, 30],
  [3, 43],
  [4, 60],
  [5, 82],
  [6, 95],
  [7, 81],
  [8, 48],
  [9, 41],
  [10, 30],
  [11, 33],
  [12, 33],
  [13, 53],
  [14, 75],
  [15, 85],
  [16, 77],
  [17, 62],
  [18, 64],
  [19, 54],
  [20, 46],
  [21, 10]],
 'Maxwell Food Centre': [[0, 9],
  [1, 20],
  [2, 34],
  [3, 51],
  [4, 66],
  [5, 92],
  [6, 101],
  [7, 84],
  [8, 52],
  [9, 42],
  [10, 30],
  [11, 33],
  [12, 36],
  [13, 57],
  [14, 82],
  [15, 90],
  [16, 77],
  [17, 64],
  [18, 69],
  [19, 56],
  [20, 48],
  [21, 13]],
 'Hong Lim Food Centre': [[0, 6],
  [1, 13],
  [2, 13],
  [3, 21],
  [4, 31],
  [5, 40],
  [6, 44],
  [7, 36],
  [8, 22],
  [9, 21],
  [10, 17],
  [11, 18],
  [12, 18],
  [13, 23],
  [14, 37],
  [15, 42],
  [16, 34],
  [17, 27],
  [18, 29],
  [19, 23],
  [20, 23],
  [21, 4]],
 'Old Airport Road Food Centre': [[0, 6],
  [1, 20],
  [2, 25],
  [3, 36],
  [4, 51],
  [5, 69],
  [6, 75],
  [7, 66],
  [8, 36],
  [9, 34],
  [10, 27],
  [11

In [14]:
def order_time_generation(t1, t2, num_orders):
    '''
    Generate random order create time in time interval [t1, t2]
    Inputs:
    - t1: start time
    - t2: end time
    - num_orders: number of orders create in [t1, t2]
    Output:
    - list of order create time
    '''
    num = 0
    output = []
    
    while num < num_orders:
        integer = random.randint(0, (t2 - t1).total_seconds())
        step = timedelta(seconds=integer)
        order_time = t1 + step
        output.append(order_time)
        num = num + 1
    
    output.sort()
    
    return [i.strftime('%Y-%m-%d %H:%M:%S') for i in output]

In [15]:
def order_id_generation(time_list):
    '''
    Input:
    - time_list: list of order create time, output of Function order_time_generation
    Output:
    - list of order id
    '''
    output = []
    replace_char = "-: "
    
    for o in time_list:
        for c in replace_char:
            o = o.replace(c,"")
        output.append(o)
    
    return output

In [16]:
def order_location_generation(order_id_list, restaurant):
    '''
    Input:
    - order_id_list
    - restaurant
    Output:
    - list of orders location
    '''
    o_location_list = random.sample(restaurant_customer_dict[restaurant], len(order_id_list))
    return o_location_list

In [17]:
restaurants_select = ['Ghim Moh Market & Food Centre',
                      'Bukit Timah Market & Food Centre',
                      'Alexandra Village Food Centre']

In [18]:
def instance_generation(restaurants):
    '''
    Inputs:
    - restaurants： select restaurants
    Output:
    - Instance
    '''
    instance_df = pd.DataFrame(columns=['order_id', 'demand', 'creation_time', 'committed_completion_time', 
                                        'load_time', 'unload_time', 'pickup_id', 'delivery_id'])
    
    # one instance (from 6 AM to 22:30 PM)
    for time_id in time_period_id:

        # for each time period
        start = datetime.strptime(time_id_period_dict[time_id], "%Y-%m-%d %H:%M:%S")
        end = datetime.strptime(time_id_period_dict[time_id], "%Y-%m-%d %H:%M:%S") + timedelta(minutes=45)

        # for each restaurant
        for r in restaurants:
            small_df = pd.DataFrame()

            order_time_list = order_time_generation(start, end, num_orders=restaurant_order_num[r][time_id][1])
            order_id_list = order_id_generation(order_time_list)
            order_location_list = order_location_generation(order_id_list, r)

            small_df['order_id'] = order_id_list
            small_df['creation_time'] = order_time_list
            small_df['restaurant_name'] = [r] * len(order_id_list)
            small_df['customer_name'] = [o[0] for o in order_location_list]
            small_df['pick_up_coordinate'] = [restaurant_coordinate_dict[r]] * len(order_id_list)
            small_df['delivery_coordinate'] = [o[1] for o in order_location_list]

            instance_df = instance_df.append(small_df, ignore_index=True)
    
    # update instance dataframe
    instance_df['demand'] = 1
    instance_df['load_time'] = 60
    instance_df['unload_time'] = 60
    
    instance_df['creation_time'] = pd.to_datetime(instance_df['creation_time'])
    instance_df = instance_df.sort_values(by='creation_time').reset_index()
    instance_df['committed_completion_time'] = instance_df['creation_time'] + timedelta(minutes=45)
    
    instance_df['pickup_id'] = instance_df['restaurant_name']
    instance_df['delivery_id'] = instance_df['customer_name']
    
    instance_df.drop(['index'], axis=1, inplace=True)
    
    for index, row in instance_df.iterrows():
        instance_df.at[index, 'order_id'] = row['order_id'] + str(index).zfill(5)

    return instance_df

In [19]:
instance = instance_generation(restaurants=restaurants_select)
instance

NameError: name 'restaurant_customer_dict' is not defined